### Ноутбук для экспериментов и измерения метрик

Работает с окружением vtb или vtb_dgx_server

Также можно использовать скрипт без визуализации (vis=False)

```bash
source activate vtb
cd VTB_OCR/
PYTHONPATH=. python scripts/evaluate.py
```

In [1]:
import os
import torch
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import nip
from albumentations import NoOp, RandomRotate90

INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.4 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
os.chdir('../')
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
DEVICE = 'cuda'

### Импорт инструментов из библиотеки

In [3]:
from lib.data.ocr_dataset import CollectedDataset
from lib.models.detection.YOLOX.yolox_predictor import YOLOXPredictor
from lib.evaluation.general_evaluator import GeneralEvaluator
from lib.models.recognition.rec_predictor import RecPredictor
from lib.models.detection.seg_predictor import SegPredictor
from lib.models.dewarping.rotation_predictor import RotPredictor
from lib.models.dewarping.convnet import ConvNet
from lib.utils import register_everything, load_model

register_everything()

/home/konovalenko_f/vtb_project/VTB_OCR/lib/core/trainer.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/konovalenko_f/vtb_project/VTB_OCR/lib/generation/tools/layer_morph.py:348: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_inverse_map(warp_map):
/home/konovalenko_f/vtb_project/VTB_OCR/lib/generation/generators/warp_image_generator.py:14: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be

### Пути к датасетам и моделям

**Датасеты**

Используется 3 датасета
- collected_v1.3 (датасет сканов паспортов РФ)
- docs_new (датасет сканов паспортов РФ и фото разных документов)
- different_docs_2025 (датасет сканов и фото разных документов)

**Модели**

В текущей версии 1 тип детектора и 1 тип OCR модели (наиболее перспективные)
- YOLOX
- модель на базе tiny ViTSTR


In [4]:
td_data_pth = '/data/shared/VTB_OCR/datasets/different_docs_2025'
# passp_data_pth = '/data/shared/VTB_OCR/datasets/docs_new'
# data_pth = '/data/shared/VTB_OCR/datasets/open_passports/collected_v1.3'

yolo_b = '/home/belyaev_v/experiments/yolox_ddi+birth_0.67_0.75_lr_0.001_size_960_nt/model_last.pth'
yolo_s = '/home/belyaev_v/experiments/yolox_ddi+snils_0.67_0.75_lr_0.001_size_960/model_last.pth'
rec_pth = '/data/shared/VTB_OCR/experiments/ocr/vitstr/backbone_vit_tiny_4_8_cat_new_set_from_pre/model_850.pth'
rec_pth_n = '/data/shared/VTB_OCR/experiments/ocr/sviptr/VIPTRv2T_ch_in_3_CTC_cat_new_set_noise/model_3000.pth'
rec_pth_new = '/data/shared/VTB_OCR/experiments/ocr/sviptr/VIPTRv2T_ch_CTC_gray_mixed_docs/model_last.pth'
rec_pth_new_cyr = '/data/shared/VTB_OCR/experiments/ocr/sviptr/VIPTRv2T_ch_CTC_gray_russian_docs/model_2500.pth'
dew_pth = "/data/shared/VTB_OCR/experiments/dewarping/convnet_rotation/rotation.pth"

# UNets
unet_b_no_templates = "/data/shared/VTB_OCR/experiments/detection/unet_ddi+birtonly_aug_nt_4_8_lr_0.001_size_1280/model_last.pth"
unet_s_no_templates = "/data/shared/VTB_OCR/experiments/detection/unet_ddi+snilsonly_aug_4_8_lr_0.001_size_1280/model_last.pth"

vocab = nip.load('configs/recognition/datasets/alphabet.nip')
vocab_ru = nip.load('configs/recognition/datasets/alphabet_ru.nip')

### Инициализация датасета с разметкой по словам

- path - путь к папке датасета
- gray - одноканальное изображение или нет
- mode - для e2e пайплайна - end2end. В режиме recognition датасет обрезает боксы по GT разметке и отдает отдельные слова
если нет OCR разметки и нужен предикт без расчета метрик - использовать detection
- transform - для e2e пайплайна - None (на выходе из датасета изображение в формате np.array)
- convert_to_rgb - True
- skip_empty_text - True
- skip_template_words - True (для формата данных ВТБ)
- skip_vertical_text - True (для разметки где есть флаг вертикальности бокса)
- box_expand - (-1, -1) - коэффициенты расширения GT боксов
- output_shape - (960, 640) - размер изображения на выходе из датасета (H, W)
- subset - поддатасет: для collected_v1.3 ('all') для docs_new - ('scans', 'photos')
- split - сплит: для collected_v1.3 ('valid') для docs_new - не указывается
- vocab - None - словарь
- name - имя датасета

In [5]:
test_set = CollectedDataset(path=td_data_pth,
                      gray=False,
                      mode='end2end',
                      transform=None,
                      convert_to_rgb=True,
                      skip_empty_text=False,
                      skip_template_words=False,
                      skip_vertical_text=True,
                      box_expand=(-1, -1),
                      output_shape=(960, 640),
                      subset='snils',
                      split="valid",
                      vocab=vocab)

### Инициализация деварпера

В данном случае - корректор угла поворота для повернутых или перевернутых изображений
ConvNet модель и обертка для нее

- dew_path - путь к модели
- name - имя модели

In [6]:
net = ConvNet()
rot = load_model(path=dew_pth, model=net)
dewarper = RotPredictor(model=rot,
                        device=DEVICE)

/data/shared/CONDAENVS/vtb_dgx_server/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/data/shared/CONDAENVS/vtb_dgx_server/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Инициализация детектора

Unet модель и обертка для нее

- det_path - путь к модели
- gray - одноканальное изображение или нет - для Unet - True
- name - имя модели

In [7]:
# yolo = load_model(yolo_b, device=DEVICE)
# detector = YOLOXPredictor(yolo,
#                          device=DEVICE,
#                          gray=False,
#                          img_shape=(960, 960),
#                          name=yolo_b.split("/")[-2],
#                          )

In [8]:
unet = load_model(unet_s_no_templates, device=DEVICE)
detector = SegPredictor(unet,
                         device=DEVICE,
                         gray=False,
                         img_shape=(1280, 832),
                         name=unet_s_no_templates.split("/")[-2],
                         )

### Инициализация OCR

ViT based модель и обертка для нее

- rec_path - путь к модели
- gray - одноканальное изображение или нет - для VitSTR - False
- input_shape - размер бокса на входе в OCR модель
- name - имя модели

In [9]:
ocr = load_model(rec_pth_new_cyr)
recognizer = RecPredictor(model=ocr,
                         gray=True,
                         device=DEVICE,
                         name=rec_pth_new_cyr.split("/")[-2],
                         input_shape=(32, 256),
                         )

INFO:lib.models.recognition.sviptr.sviptr:No sequence_modeling module specified


### Инициализация Эвалюатора

- dewarper - модель деварпинга
- detector - модель детекции
- recognizer - OCR - может быть None, в этом случае распознавание символов не производится, метрики не считаются
- onestage_predictor - one stage модель
- entity_matcher - модель мэтчинга сущностей
- corrector - модель корректировки сущностей
- test_dataset - датасет
- skip_num - пропускаем ли номер паспорта (GT бокс, содержащий только цифры и пробелы) при расчете TD и OCR метрик
- skip_serv - пропускаем ли сервисные поля при расчете TD и OCR метрик
- skip_mrz - пропускаем ли MRZ при расчете TD и OCR метрик
- correct_ocr - корректируем или нет предсказания OCR модели
- save_crops - сохранение нарезанных боксов (если True - в корневой папке создается папка word_crops куда сохраняются нарезанные боксы и результаты распознавания в них.)
- save_path - путь для сохранения результатов OCR в формате .csv
- archive - путь для архивирования результатов эксперимента
- comment - комментарий
- timeout - задержка между изображениями если хочется подробнее рассмотреть. Если не 0 - то вывод ячейки не очищается
- stop - на каком кадре останавливаемся
- vis - отрисовка результатов TD (если мэтчинг не задан) или деварпинга и мэтчинга (если мэтчинг задан) в ноутбуке

#### Пример TD + OCR

In [10]:
general = GeneralEvaluator(dewarper=None,
                           detector=detector,
                           recognizer=recognizer,
                           onestage_predictor=None,
                           entity_matcher=None,
                           corrector=None,
                           test_dataset=test_set,
                           skip_num=False,
                           skip_template_words=True,
                           skip_mrz=False,
                           save_crops=False,
                           correct_ocr=True,
                           device=DEVICE, 
                           save_path=None,
                           archive=None,
                           comment='snils',
                           timeout=0,
                           stop=None)

In [11]:
result, metric = general.evaluate(vis=False)

Processing images: 100%|██████████| 36/36 [00:09<00:00,  3.70it/s]

Batch 35 processed
Batch processing time (dewarping) - 0.0 seconds
Batch processing time (one stage) - 0.0 seconds
Batch processing time (detection) - 0.0941 seconds
Batch processing time (recognition) - 0.016 seconds
Batch processing time (entity matching) - 0.0 seconds
____________________

____________________

Mean metric IOU - 0.806
____________________

Mean metric AVERAGE_PRECISION@60-95 - 0.315
____________________

Mean metric AVERAGE_RECALL@60-95 - 0.684
____________________

Mean metric AVERAGE_FSCORE@60-95 - 0.612
____________________

Mean metric PRECISION@IOU0.6 - 0.714
____________________

Mean metric RECALL@IOU0.6 - 0.882
____________________

Mean metric FSCORE@IOU0.6 - 0.789
____________________

Mean metric WORD_ACCURACY - 0.882
____________________

Mean metric SIMILARITY - 0.882
____________________

Mean metric CHARACTER_ERROR - 8.16
____________________

Mean metric LEVENSHTEIN_DISTANCE - 0.118
____________________



EVALUATION COMPLETED
Result saved to results_None file
Metrics saved to metrics_None file
Experiment metrics saved to None file


In [12]:
metric

{'date': '2025-02-19 13:38:16',
 'dataset': 'different_docs_2025',
 'comment': 'snils',
 'detector': 'unet_ddi+snilsonly_aug_4_8_lr_0.001_size_1280',
 'ocr_recognizer': 'VIPTRv2T_ch_CTC_gray_russian_docs',
 'dewarper_time': 0.0,
 'onestage_time': 0.0,
 'detector_time': 0.0943,
 'recognizer_time': 0.0272,
 'matching_time': 0.0,
 'device': device(type='cuda'),
 'iou': 0.752,
 'average_precision@60-95': 0.374,
 'average_recall@60-95': 0.579,
 'average_fscore@60-95': 0.542,
 'precision@IoU0.6': 0.796,
 'recall@IoU0.6': 0.888,
 'fscore@IoU0.6': 0.83,
 'word_accuracy': 0.844,
 'similarity': 0.845,
 'character_error': 12.487,
 'levenshtein_distance': 0.112}

In [13]:
result.head()

,gt,pred,filename
0,ГОДА,ГОДА,0030
0,2004,2004,0030
0,ЯНВАРЯ,ЯНВАРЯ,0030
0,19,19,0030
0,МУЖСКОЙ,МУЖСКОЙ,0030


In [14]:
# pd.set_option('display.max_columns', None)
# pd.read_csv('/data/shared/VTB_OCR/experiments/history.csv', index_col=0).tail()